# **Điểm chuẩn - Preprocessing**

Preprocessing data điểm chuẩn đã crawl về từ web [huongnghiep.hocmai.vn](https://huongnghiep.hocmai.vn/diem-chuan/)

## **Intro:**
- Bộ dữ liệu raw là các file csv như sau:  

    ```
    CRAWLER-HOCMAI  
    └── data  
        └── full  
            ├── diemchuan2018_demo.csv  
            ├── diemchuan2019_full.csv  
            ├── diemchuan2020_full.csv  
            ├── diemchuan2021_full.csv  
            ├── diemchuan2022_full.csv  
            ├── diemchuan2023_full.csv  
    ```
- Tương ứng mỗi file là dữ liệu điểm chuẩn của 1 năm.
    - 2 file `full` là dữ liệu đã được crawled đầy đủ.
    - 4 file `demo` là dữ liệu ***gần như*** đầy đủ, vì trong quá trình crawl có bị missing một vài trường. Sẽ cố gắng cập nhật đầy đủ sớm hơn.
- Cấu trúc dữ liệu mỗi file gồm 7 cột `university_code`, `university_name`, `major_code`, `major_name`, `subject_group`, `point`, `note`, `year`. 

    Một mẫu dữ liệu minh hoạ như sau: 
    
| university_code | university_name                 | major_code | major_name               | subject_group       | point | note             | year |
|-----------------|---------------------------------|------------|--------------------------|---------------------|-------|------------------|------|
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7140234    | Sư phạm Tiếng Trung Quốc | D01, D15, D04, D45  | 24.85 | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7140233    | Sư phạm Tiếng Pháp       | D01, D15, D03, D44  | 19    | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7220204    | Ngôn ngữ Trung Quốc      | D01, D15, D04, D45  | 23    | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7220210    | Ngôn ngữ Hàn Quốc        | D01, D14, D15       | 22.5  | Tốt nghiệp THPT  | 2023 |



## **Target**
- Kiểm tra tính đầy đủ của dữ liệu (về số trường đại học).

- Kiểm tra thiếu dữ liệu của từng dòng.

- Kiểm tra trùng lặp.

- **Phân tách dữ liệu cho từng loại điểm**


## **Start here**

### **import các thư viện**

In [1]:
import pandas as pd
import numpy as np
import os

### **Chỉ định đường dẫn thư mục**

In [2]:
# Directory containing the CSV files
directory = './data/full'


### **Đọc các file csv vào df**

In [3]:
# Dictionary to store DataFrames
dfs = {}
raw = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct full file path
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Store the DataFrame in the dictionary with the filename (without .csv) as the key
        dfs[filename.split('_')[0]] = df
        raw.append(df)

# Display the keys of the dictionary to confirm the DataFrames are loaded
dfs.keys()


dict_keys(['diemchuan2018', 'diemchuan2019', 'diemchuan2020', 'diemchuan2021', 'diemchuan2022', 'diemchuan2023'])

In [4]:
shapes = {name: df.shape for name, df in dfs.items()}
shapes

{'diemchuan2018': (3495, 8),
 'diemchuan2019': (5706, 8),
 'diemchuan2020': (4924, 8),
 'diemchuan2021': (6561, 8),
 'diemchuan2022': (7991, 8),
 'diemchuan2023': (7947, 8)}

In [5]:
full_df = pd.concat(raw, ignore_index=True)
full_df.head()

,university_code,university_name,major_code,major_name,subject_group,point,note,year
0,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018
1,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7520207,Kỹ thuật điện tử - viễn thông,"A00, A01",17.00,(Cơ sở đào tạo phía Nam) TTNV <= 7,2018
2,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7329001,Công nghệ đa phương tiện,"A00, A01, D01",19.25,(Cơ sở đào tạo phía Nam) TTNV <= 8,2018
3,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7480201,Công nghệ thông tin,"A00, A01",20.25,(Cơ sở đào tạo phía Nam) TTNV <= 8,2018
4,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7340101,Quản trị kinh doanh,"A00, A01, D01",18.20,(Cơ sở đào tạo phía Nam) TTNV <= 9,2018


In [6]:
full_df.shape

(36624, 8)

#### Đổi tên các cột lại thành tiếng Việt

In [7]:
# Dictionary to map old column names to new column names
column_mapping = {
    'university_code': 'Mã Trường',
    'university_name': 'Tên Trường',
    'major_code': 'Mã Ngành',
    'major_name': 'Tên Ngành',
    'subject_group': 'Tổ hợp',
    'point': 'Điểm chuẩn',
    'note': 'Ghi chú',
    'year': 'Năm'
}

# Rename columns for each DataFrame in the dictionary
for name, df in dfs.items():
    df.rename(columns=column_mapping, inplace=True)

full_df.rename(columns=column_mapping, inplace=True)
full_df.head()

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm
0,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018
1,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7520207,Kỹ thuật điện tử - viễn thông,"A00, A01",17.00,(Cơ sở đào tạo phía Nam) TTNV <= 7,2018
2,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7329001,Công nghệ đa phương tiện,"A00, A01, D01",19.25,(Cơ sở đào tạo phía Nam) TTNV <= 8,2018
3,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7480201,Công nghệ thông tin,"A00, A01",20.25,(Cơ sở đào tạo phía Nam) TTNV <= 8,2018
4,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7340101,Quản trị kinh doanh,"A00, A01, D01",18.20,(Cơ sở đào tạo phía Nam) TTNV <= 9,2018


### **Khám phá dữ liệu**

#### 1. Mỗi dòng có ý nghĩa gì?

In [8]:
full_df.sample(5)

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm
5089,EPU,Đại Học Điện Lực,7340101,Quản trị kinh doanh,"A00, A01, D01, D07",15.00,NaN,2019
8201,HNUE,Đại Học Sư Phạm Hà Nội 2,7220204,Ngôn ngữ Trung Quốc,"A01, D01, D04, D11",23.00,NaN,2019
17714,SGU,Đại Học Sài Gòn,7210205,Thanh nhạc,N02,20.50,Điểm thi TN THPT,2021
29159,IUH,Đại Học Công Nghiệp TPHCM,7340301,Kế toán,DGNLHCM,765.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023
25346,HAUI,Đại Học Công Nghiệp Hà Nội,7540101,Công nghệ thực phẩm,"A00, B00, D07, XDHB",23.75,Tiêu chí phụ 1 Điểm Toán ≥ 8\nTiêu chí phụ 2 T...,2022


Mỗi dòng là thông tin về điểm chuẩn của một ngành, tại một trường, vào một năm cụ thể.

##### Kiểm tra trùng lặp

In [9]:
if len(full_df[full_df.duplicated()]) == 0:
    print(f'Không có dòng nào trùng lặp trong dữ liệu.')
else:
    print(f'Tồn tại {len(full_df[full_df.duplicated()])} dòng bị trùng lặp trong dữ liệu.')

Tồn tại 14 dòng bị trùng lặp trong dữ liệu.


Xuất các dòng trùng lặp ra file csv (Để coi chơi thôi chứ cũng drop à :v)

In [10]:
full_df[full_df.duplicated()].to_csv('./data/dup.csv',  encoding='utf-8-sig')

Loại bỏ các dòng trùng

In [11]:
full_df.drop_duplicates(inplace=True)

Kiểm tra lại

In [12]:
if len(full_df[full_df.duplicated()]) == 0:
    print(f'Không có dòng nào trùng lặp trong dữ liệu.')
else:
    print(f'Tồn tại {len(full_df[full_df.duplicated()])} dòng bị trùng lặp trong dữ liệu.')

Không có dòng nào trùng lặp trong dữ liệu.


#### 2. Mỗi cột có ý nghĩa gì?

Các cột và thông tin tương ứng:
- `Mã Trường`: Tên viết tắt của trường (Không phải mã trên hệ thống của bộ).

- `Tên Trường`: Tên của Trường đại học.

- `	Mã Ngành`: Mã ngành tuyển sinh.

- `	Tên Ngành`: Tên ngành tuyển sinh.

- `	Tổ hợp`: Các tổ hợp môn xét tuyển.

- `	Điểm chuẩn`: Điểm chuẩn tương ứng với tổ hợp.

- `	Ghi chú`: Ghi chú thêm (thường là thông tin về điều kiện bổ sung, hoặc phương thức xét tuyển).

- `	Năm`: Năm tuyển sinh.

#### 3. Mỗi cột hiện có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp cần xử lý tiếp hay không?

In [13]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36610 entries, 0 to 36623
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Mã Trường   36610 non-null  object 
 1   Tên Trường  36610 non-null  object 
 2   Mã Ngành    36610 non-null  object 
 3   Tên Ngành   36610 non-null  object 
 4   Tổ hợp      36587 non-null  object 
 5   Điểm chuẩn  36610 non-null  float64
 6   Ghi chú     24764 non-null  object 
 7   Năm         36610 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 2.5+ MB


Các cột có kiểu dữ liệu phù hợp.

#### 4. Với mỗi cột, các giá trị (dạng số, dạng phân loại) được phân bố như thế nào?

##### 4.1. Các cột dạng số

In [14]:
numerical_columns = full_df.select_dtypes(include=['int64', 'float64']).columns
print("Numerical columns:", numerical_columns)

def calculate_missing_ratio(df):
    # Tính tỷ lệ dữ liệu thiếu cho từng cột
    missing_ratios = df.isna().mean() * 100

    # Trả về Series chứa tỷ lệ dữ liệu thiếu cho từng cột
    return missing_ratios

# Gọi hàm tính missing ratio và in kết quả
missing_ratios = calculate_missing_ratio(full_df[numerical_columns])
print('Missing ratio:')
print(missing_ratios)
full_df.describe()

Numerical columns: Index(['Điểm chuẩn', 'Năm'], dtype='object')
Missing ratio:
Điểm chuẩn    0.0
Năm           0.0
dtype: float64


,Điểm chuẩn,Năm
count,36610.000000,36610.000000
mean,75.582430,2020.919312
std,184.936156,1.642015
min,0.000000,2018.000000
25%,16.000000,2019.000000
50%,20.000000,2021.000000
75%,25.250000,2022.000000
max,1035.000000,2023.000000


In [15]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Năm' hoặc cột 'Điểm chuẩn' (nếu có)
missing_rows = full_df[full_df['Năm'].isna() | full_df['Điểm chuẩn'].isna()]
missing_rows

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm


In [16]:
full_df.drop(missing_rows.index, inplace=True)
full_df.reset_index(drop=True, inplace=True)

In [17]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Năm' hoặc cột 'Điểm chuẩn' (nếu có)
missing_rows = full_df[full_df['Năm'].isna() | full_df['Điểm chuẩn'].isna()]
missing_rows

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm


Thấy min của điểm chuẩn là 0 (kỳ lạ) nên ta sẽ xử lý các dòng có điểm chuẩn = 0 một chút.


In [18]:
full_df[full_df['Điểm chuẩn']==0]

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm
25,TDTU,Đại Học Tôn Đức Thắng,K7310630Q,Quản lý du lịch và giải trí,"A01, D01, C00, C01",0.0,(song bằng 2+2) - Chương trình liên kết Đại họ...,2018
26,TDTU,Đại Học Tôn Đức Thắng,K7340101,Quản trị kinh doanh,"A00, A01, D01",0.0,(song bằng 2+2) - Chương trình liên kết Đại họ...,2018
27,TDTU,Đại Học Tôn Đức Thắng,K7340201,Tài chính,"A00, A01, D01, D07",0.0,(song bằng 2+2) -Chương trình liên kết Đại học...,2018
28,TDTU,Đại Học Tôn Đức Thắng,K7340301,Kế toán,"A00, A01, D01, C01",0.0,(song bằng 3+1) - Chương trình liên kết Đại họ...,2018
66,HPMU,Đại Học Y Dược Hải Phòng,7720101A,Y khoa,A00,0.0,A,2018
...,...,...,...,...,...,...,...,...
36247,NCTU,Đại học Nam Cần Thơ,7520212,Kỹ thuật y sinh,"A00, B00, A01, A02, XDHB",0.0,NaN,2023
36249,NCTU,Đại học Nam Cần Thơ,7520320,Kỹ thuật môi trường,"A00, B00, A02, C08, XDHB",0.0,NaN,2023
36251,NCTU,Đại học Nam Cần Thơ,7720802,Quản trị bệnh viện,"B00, B03, C01, C02, XDHB",0.0,NaN,2023
36253,NCTU,Đại học Nam Cần Thơ,7810202,Quản trị nhà hàng và dịch vụ ăn uống,"A00, A01, D01, C00, XDHB",0.0,NaN,2023


Ta thấy có rất nhiều ngành có điểm chuẩn =0, tuy nhiên, một số sẽ có chú thích riêng. Vậy, đối với các ngành có điểm chuẩn =0 và không có chú thích thì mình xem như năm đó không có tuyển sinh. 
> => Mình loại bỏ các dòng này.

In [19]:
diemchuan_0 = full_df[(full_df['Điểm chuẩn']==0) & (full_df['Ghi chú'].isna())]
diemchuan_0

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm
2101,VNUHCM-USSH,USSH - Đại Học Khoa Học Xã Hội và Nhân Văn – Đ...,7810103,Quản trị dịch vụ du lịch và lữ hành,"C00, D01",0.0,NaN,2018
2217,VNU - SMP,SMP - Đại học Y Dược - Đại học Quốc Gia Hà Nội,7720601,Kỹ thuật xét nghiệm y học,B00,0.0,NaN,2018
2218,VNU - SMP,SMP - Đại học Y Dược - Đại học Quốc Gia Hà Nội,7720602,Kỹ thuật hình ảnh y học,B00,0.0,NaN,2018
2249,VNU-HUS,HUS - Đại Học Khoa Học Tự Nhiên – Đại Học Quốc...,QHT14,Khoa học đất,"A00, B00, A01, D07",0.0,NaN,2018
2262,VNU-HUS,HUS - Đại Học Khoa Học Tự Nhiên – Đại Học Quốc...,QHT46,Công nghệ kỹ thuật môi trường,"A00, B00, A01, D07",0.0,NaN,2018
...,...,...,...,...,...,...,...,...
36247,NCTU,Đại học Nam Cần Thơ,7520212,Kỹ thuật y sinh,"A00, B00, A01, A02, XDHB",0.0,NaN,2023
36249,NCTU,Đại học Nam Cần Thơ,7520320,Kỹ thuật môi trường,"A00, B00, A02, C08, XDHB",0.0,NaN,2023
36251,NCTU,Đại học Nam Cần Thơ,7720802,Quản trị bệnh viện,"B00, B03, C01, C02, XDHB",0.0,NaN,2023
36253,NCTU,Đại học Nam Cần Thơ,7810202,Quản trị nhà hàng và dịch vụ ăn uống,"A00, A01, D01, C00, XDHB",0.0,NaN,2023


In [20]:
shape = full_df.shape

# Drop các dòng có trong diemchuan_0
full_df.drop(diemchuan_0.index, inplace=True)

# Reset lại chỉ số sau khi drop
full_df.reset_index(drop=True, inplace=True)

print(f"Dữ liệu trước xử lý có {shape[0]} dòng và {shape[1]} cột.")
print(f"Dữ liệu   sau xử lý có {full_df.shape[0]} dòng và {full_df.shape[1]} cột.")

Dữ liệu trước xử lý có 36610 dòng và 8 cột.
Dữ liệu   sau xử lý có 35976 dòng và 8 cột.


##### 4.2. Các cột dạng phân loại

In [21]:
categorical_columns = full_df.select_dtypes(include=['object']).columns
categorical_columns

Index(['Mã Trường', 'Tên Trường', 'Mã Ngành', 'Tên Ngành', 'Tổ hợp',
       'Ghi chú'],
      dtype='object')

In [22]:
def calculate_column_profile(column):
    missing_ratio = len(column[column.isna()]) * 100 / len(column)
    num_diff_vals = len(column.dropna().unique())
    diff_vals = column.dropna().unique()
    return pd.Series([missing_ratio, num_diff_vals, diff_vals], index=['missing_ratio', 'num_diff_vals', 'diff_vals'])

cate_col_profiles_df = full_df[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Ghi chú
missing_ratio,0.0,0.0,0.0,0.0,0.013898,31.165221
num_diff_vals,255,323,2585,667,2476,5200
diff_vals,"[UTC, PTIT, HUFLIT, UTC2, TDTU, TLU, NUTE, HAU...","[Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc), ...","[GHA-23, 7520207, 7329001, 7480201, 7340101, 7...","[Công nghệ thông tin, Kỹ thuật điện tử - viễn ...","[A00, A01, D07, A00, A01, A00, A01, D01, A01, ...","[7.4, (Cơ sở đào tạo phía Nam) TTNV <= 7, (Cơ ..."


(Lại) Thấy `Tổ hợp` có missing, trong khi dữ liệu này đáng lẽ không nên miss.  Mình in ra xem thử

In [23]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Mã Ngành' hoặc cột 'Tổ hợp' (nếu có)
missing_rows = full_df[full_df['Tổ hợp'].isna()]
missing_rows

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm
1732,TDU,Đại học Thành Đô,6220206,Tiếng Anh,NaN,0.0,Tốt nghiệp THPT,2018
1733,TDU,Đại học Thành Đô,6220213,Tiếng Nhật,NaN,0.0,Tốt nghiệp THPT,2018
1734,TDU,Đại học Thành Đô,6810103,Hướng dẫn du lịch,NaN,0.0,Tốt nghiệp THPT,2018
1735,TDU,Đại học Thành Đô,6810201,Quản trị khách sạn,NaN,0.0,Tốt nghiệp THPT,2018
3481,VNUA,Học Viện Nông Nghiệp Việt Nam,7140215,Sư phạm Kỹ thuật nông nghiệp,NaN,0.0,Tuyển thẳng đối với thí sinh đạt học lực giỏi ...,2019


Thấy toàn bộ missing là tuyển thẳng, nên Replace "Tuyển thẳng"

In [24]:
full_df.loc[full_df['Tổ hợp'].isna(), 'Tổ hợp'] = 'Tuyển thẳng'

In [25]:
cate_col_profiles_df = full_df[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Ghi chú
missing_ratio,0.0,0.0,0.0,0.0,0.0,31.165221
num_diff_vals,255,323,2585,667,2477,5200
diff_vals,"[UTC, PTIT, HUFLIT, UTC2, TDTU, TLU, NUTE, HAU...","[Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc), ...","[GHA-23, 7520207, 7329001, 7480201, 7340101, 7...","[Công nghệ thông tin, Kỹ thuật điện tử - viễn ...","[A00, A01, D07, A00, A01, A00, A01, D01, A01, ...","[7.4, (Cơ sở đào tạo phía Nam) TTNV <= 7, (Cơ ..."


### **Tách cột Tổ hợp**

In [26]:
# Tách cột "Tổ hợp" theo dấu phẩy và tạo các dòng mới
#df_expanded = full_df.assign(subject_group=full_df['Tổ hợp'].str.split(', ')).explode('Tổ hợp')
df_expanded = full_df.assign(subject_group=full_df['Tổ hợp'].str.split(', ')).explode('subject_group')

df_expanded.reset_index(drop=True, inplace=True)
df_expanded

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,subject_group
0,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018,A00
1,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018,A01
2,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018,D07
3,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7520207,Kỹ thuật điện tử - viễn thông,"A00, A01",17.00,(Cơ sở đào tạo phía Nam) TTNV <= 7,2018,A00
4,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7520207,Kỹ thuật điện tử - viễn thông,"A00, A01",17.00,(Cơ sở đào tạo phía Nam) TTNV <= 7,2018,A01
...,...,...,...,...,...,...,...,...,...
111899,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM
111900,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM
111901,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM
111902,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM


In [27]:
condition2 = (df_expanded['Ghi chú'].str.contains('ao đẳng', na=False))
df_expanded.drop(df_expanded[condition2].index, inplace=True)

# Lọc và xóa các dòng có subject_group là 'XDHB' và 'ọc bạ' tồn tại trong Ghi chú
condition = (df_expanded['subject_group'] == 'XDHB') & (df_expanded['Ghi chú'].str.contains('ọc bạ', na=False))
df_expanded.drop(df_expanded[condition].index, inplace=True)


condition3 = (df_expanded['subject_group'] == 'XDHB') & (df_expanded['Tổ hợp'].str.contains(', XDHB', na=False))
df_expanded.drop(df_expanded[condition3].index, inplace=True)

# Đặt lại chỉ mục của full_df và lưu thay đổi này vào DataFrame gốc
df_expanded.reset_index(drop=True, inplace=True)
df_expanded



,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,subject_group
0,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018,A00
1,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018,A01
2,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018,D07
3,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7520207,Kỹ thuật điện tử - viễn thông,"A00, A01",17.00,(Cơ sở đào tạo phía Nam) TTNV <= 7,2018,A00
4,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7520207,Kỹ thuật điện tử - viễn thông,"A00, A01",17.00,(Cơ sở đào tạo phía Nam) TTNV <= 7,2018,A01
...,...,...,...,...,...,...,...,...,...
104957,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM
104958,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM
104959,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM
104960,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM


In [28]:
categorical_columns = df_expanded.select_dtypes(include=['object']).columns
cate_col_profiles_df = df_expanded[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Ghi chú,subject_group
missing_ratio,0.0,0.0,0.0,0.0,0.0,30.649187,0.0
num_diff_vals,255,323,2569,666,2471,5072,279
diff_vals,"[UTC, PTIT, HUFLIT, UTC2, TDTU, TLU, NUTE, HAU...","[Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc), ...","[GHA-23, 7520207, 7329001, 7480201, 7340101, 7...","[Công nghệ thông tin, Kỹ thuật điện tử - viễn ...","[A00, A01, D07, A00, A01, A00, A01, D01, A01, ...","[7.4, (Cơ sở đào tạo phía Nam) TTNV <= 7, (Cơ ...","[A00, A01, D07, D01, D14, D15, D11, C00, C01, ..."


In [29]:
# df_expanded.to_csv("./data/preprocessing/full.csv",  encoding='utf-8-sig', index=False)

In [30]:
df_expanded['Loại điểm'] = np.where(df_expanded['Ghi chú'].str.contains('Học bạ', na=False), 'Học bạ', '')
df_expanded['Loại điểm'] = np.where(((df_expanded['Tổ hợp'].str.contains('DGNL', na=False)) | (df_expanded['Tổ hợp'].str.contains('DGTD', na=False))), 'Kỳ thi riêng', 'THPTQG')
df_expanded

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,subject_group,Loại điểm
0,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018,A00,THPTQG
1,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018,A01,THPTQG
2,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,"A00, A01, D07",19.65,7.4,2018,D07,THPTQG
3,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7520207,Kỹ thuật điện tử - viễn thông,"A00, A01",17.00,(Cơ sở đào tạo phía Nam) TTNV <= 7,2018,A00,THPTQG
4,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7520207,Kỹ thuật điện tử - viễn thông,"A00, A01",17.00,(Cơ sở đào tạo phía Nam) TTNV <= 7,2018,A01,THPTQG
...,...,...,...,...,...,...,...,...,...,...
104957,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM,Kỳ thi riêng
104958,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM,Kỳ thi riêng
104959,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM,Kỳ thi riêng
104960,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,DGNLHCM,Kỳ thi riêng


In [31]:
df_expanded.drop(columns = ['Tổ hợp'], inplace=True)
df_expanded.rename(columns={'subject_group': 'Tổ hợp'}, inplace=True)
df_expanded = df_expanded[['Mã Trường', 'Tên Trường', 'Mã Ngành', 'Tên Ngành', 'Loại điểm', 'Tổ hợp', 'Điểm chuẩn', 'Ghi chú', 'Năm']]
df_expanded 

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Loại điểm,Tổ hợp,Điểm chuẩn,Ghi chú,Năm
0,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,THPTQG,A00,19.65,7.4,2018
1,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,THPTQG,A01,19.65,7.4,2018
2,UTC,Đại Học Giao Thông Vận Tải (Cơ sở Phía Bắc),GHA-23,Công nghệ thông tin,THPTQG,D07,19.65,7.4,2018
3,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7520207,Kỹ thuật điện tử - viễn thông,THPTQG,A00,17.00,(Cơ sở đào tạo phía Nam) TTNV <= 7,2018
4,PTIT,Học Viện Công Nghệ Bưu Chính Viễn Thông (phía ...,7520207,Kỹ thuật điện tử - viễn thông,THPTQG,A01,17.00,(Cơ sở đào tạo phía Nam) TTNV <= 7,2018
...,...,...,...,...,...,...,...,...,...
104957,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,Kỳ thi riêng,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023
104958,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,Kỳ thi riêng,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023
104959,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,Kỳ thi riêng,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023
104960,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,Kỳ thi riêng,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023


In [32]:
df_expanded.to_csv("./data/diemchuan_full.csv",  encoding='utf-8-sig', index=False)